##### **This notebook combines IO and control data over one dimension**

In [ ]:
import pandas as pd
import numpy as np
import warnings
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
from tqdm import tqdm
import sys
import os

import importlib

#### packages
import helper.strategy_helper as st
import helper.visualization as vz
import helper.helper as hp
import helper.file_helper as file_hp
import config.config as config

In [ ]:
config = config.config()
path = config['PATHS']

derived_path = path['derived_path']
all_tweet_data = path['all_tweet_data']

In [3]:
test_campaign = 'iran_202012'
year = '2020_12'
ops_file_path = os.path.join(all_tweet_data, 
                             year, 
                             test_campaign, 
                            f'{test_campaign}_tweets.pkl.gz')
control_file_path = os.path.join(all_tweet_data, 
                                 year, 
                                 test_campaign, 
                                 'DriversControl', 
                            f'{test_campaign}_control.pkl.gz')

data = st.read_ops_control_data(ops_file_path, 
                             control_file_path, 
                             ['ops', 'control'])
df_ops = data['ops']
df_control = data['control']

In [10]:
# df_ops.info()


In [15]:
df_ops_replies = st.filter_tweets(df_ops, 
                                  False, True, 10)

df_ops_replies = df_ops_replies[['userid', 
                                 'in_reply_to_userid', 
                                 'in_reply_to_tweetid']]

df_ops_replies = df_ops_replies.astype({
    'in_reply_to_userid': int,
    'in_reply_to_tweetid': int
})



df_control_replies = st.filter_tweets(df_control, 
                                  False, True, 10)

df_control_replies = df_control_replies[['userid', 
                                 'in_reply_to_userid', 
                                 'in_reply_to_tweetid']]

df_control_replies = df_control_replies.astype({
    'in_reply_to_userid': int,
    'in_reply_to_tweetid': int
})

print('Ops ', df_ops_replies['in_reply_to_userid'].nunique())
print('Conrol ', df_control_replies['in_reply_to_userid'].nunique())

# df_ops_replies[['userid', 'in_reply_to_userid', 'in_reply_to_tweetid']].info()

Ops  27363
Conrol  22211


In [21]:
# df_ops_replies

In [1]:
def filter(df, 
           column=['in_reply_to_userid'],
           column_to_grpby='userid',
           threshold=10):
    
    if column[0] not in df.columns:
        print(column[0], ' not in column')
        
        return
    
    if column_to_grpby not in df.columns:
        print(column_to_grpby, ' not in column')
        
        return
    
    userids = (df.groupby(column)['userid']
                   .size()
                   .to_frame('count')
                   .reset_index()
                   .query(f'count > {threshold}')
                  )[column_to_grpby]

    # df = df.loc[df[column_to_grpby].isin(userids)]
        
    return df


# df_ops_replies = filter(df_ops)

# print(len(df_ops_replies))